In [10]:
import spotipy
import credentials as cr
import pandas as pd
import ast
from spotipy.oauth2 import SpotifyClientCredentials

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Application parameters

In [2]:
# redirect_uri = 'http://localhost/'
client_id = cr.CLIENT_ID
client_secret = cr.CLIENT_SECRET
auth = cr.TOKEN
data_path = cr.DATA_PATH

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, auth=auth)

In [3]:
# def get_features(track_id):

#     return sp.audio_features(track_id)

In [4]:
def get_user_playlists():
    """
    Get the current user's spotify playlists with a limit at 50.
    ****
    returns: pandas.DataFrame playlists
    """

    # Get json playlists using spotipy api
    playlistsJson = sp.current_user_playlists(limit=50, offset=0)

    playlists = {'id': [], 'name': [], 'total_tracks': [], 'owner_id': [], 'href': []}
    
    for i, playlist in enumerate(playlistsJson['items']):
        playlists['id'].append(playlist['id'])
        playlists['name'].append(playlist['name'])
        playlists['total_tracks'].append(playlist['tracks']['total'])
        playlists['owner_id'].append(playlist['owner']['id'])
        playlists['href'].append(playlist['href'])

    print('Found playlists : ', i+1)

    return pd.DataFrame.from_dict(playlists).set_index('id')

In [5]:
def get_playlist_tracks(playlist_id):
    """
    Get the selected playlist's tracks using its id with a maximum of 100.
    ****
    returns: list of tracks of the playlist
    """

    playlistJson = sp.playlist(playlist_id)

    # Create the list to initialize the dataframe
    tracks_list = []

    for i, track in enumerate(playlistJson['tracks']['items']):
        tracks_list.append(track['track']['id'])

    # print('Found tracks : ', i+1)

    return tracks_list


In [6]:
def set_tracks_to_playlists():
    """
    Set the 100 first tracks ids to the user's playlists
    ****
    returns: dataframe with playlists and related tracks ids
    """
    playlists = get_user_playlists()

    playlists_tracks = {'id': [], 'tracks': []}

    for i, playlist in playlists.T.iteritems():
        playlists_tracks['id'].append(playlist.name)
        playlists_tracks['tracks'].append(get_playlist_tracks(playlist.name))


    df_to_merge = pd.DataFrame.from_dict(playlists_tracks).set_index('id')

    return playlists.merge(df_to_merge, left_index=True, right_index=True)


In [28]:
def get_features(track_id):
    """
    Get the selected track's features.
    ****
    returns: dict of track features"""

    track_features = sp.audio_features(track_id)    
    features = {'id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'}


    for _, feature in enumerate(track_features):
        features['id'].append(feature['id'])
        features['danceability'].append(feature['danceability'])
        features['energy'].append(feature['energy'])
        features['key'].append(feature['key'])
        features['loudness'].append(feature['loudness'])
        features['mode'].append(feature['mode'])
        features['speechiness'].append(feature['speechiness'])
        features['acousticness'].append(feature['acousticness'])
        features['instrumentalness'].append(feature['instrumentalness'])
        features['liveness'].append(feature['liveness'])
        features['valence'].append(feature['valence'])
        features['tempo'].append(feature['tempo'])
        features['duration_ms'].append(feature['duration_ms'])
        features['time_signature'].append(feature['time_signature'])

    return features

In [29]:
def get_playlist_features(playlist_df):
    """
    Get a dataframe with the features of the 100 firsts tracks of the given playlist dataframe.
    ****
    returns: dataframe of the playlist's tracks' features.
    """

    features = {'id': [], 'danceability': [], 'energy': [], 'key': [], 'loudness': [], 'mode': [], 'speechiness': [], 'acousticness': [], 'instrumentalness': [], 'liveness': [], 'valence': [], 'tempo': [], 'duration_ms': [], 'time_signature': []}

    # Iter throught playlists
    for _, playlist in playlist_df.T.iteritems():
        # Iter throught tracks ids of the playlist
        print(playlist['name'])
        for track_id in ast.literal_eval(playlist['tracks']):

            track_features = get_features(track_id)

            # Append tracks features to the features dict 
            for key in features.keys():
                features[key].append(track_features[key])


    return pd.DataFrame.from_dict(features).set_index('id')

In [30]:
playlists_df = pd.read_csv(data_path + 'playlists.csv', encoding='utf-8')

tracks_df = get_playlist_features(playlists_df)

Pop/rock kings & queens
Autoportrait
Arcades & Neons
(RPG) Leon Petersen - Don't Piss Me Off
Feel good
(RPG) Scarface - Heavy Dirty Soul
Girl Power
Alternative/Psychedelic
Musique classique
(RPG) Ashton Butler - Punkin' Shakespeare
Badass Metal
(RPG) Spooky Unicorns - Rock'N Monsters
Old Time Rock'N Roll (Kurty's playlist)
Skate It !
Favoris
Horror Circus
Ballads & Slows
(RPG) Robyn Thorn - Rockin' Girl
Top 30
(RPG) Kurt Hayward - Punk Ass
(RPG) Zak Winterscar - Broken Soldier
Accoustic
Modern Pop/Rock
Classic Rock
Ol'Retro Mood (50/60's)
Deadshot
Red Hood
Nightwing


In [31]:
#delete doubles values
tracks_df.to_csv(data_path + 'tracks.csv', encoding='utf-8')

In [ ]:
# print(sp.audio_features('3tyTt5pEWblt96j1HsXiV5'))

[{'danceability': 0.677, 'energy': 0.852, 'key': 5, 'loudness': -10.856, 'mode': 1, 'speechiness': 0.045, 'acousticness': 0.225, 'instrumentalness': 3.95e-05, 'liveness': 0.0595, 'valence': 0.93, 'tempo': 91.691, 'type': 'audio_features', 'id': '3tyTt5pEWblt96j1HsXiV5', 'uri': 'spotify:track:3tyTt5pEWblt96j1HsXiV5', 'track_href': 'https://api.spotify.com/v1/tracks/3tyTt5pEWblt96j1HsXiV5', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3tyTt5pEWblt96j1HsXiV5', 'duration_ms': 390862, 'time_signature': 4}]


In [ ]:
# playlists_df = set_tracks_to_playlists()

# playlists_df.to_csv(data_path + 'playlists.csv', encoding='utf-8')

Found playlists :  28
